In [25]:
!pip install pyspark

In [26]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.window import Window

In [27]:
# Lendo o Arquivo
spark = SparkSession.builder.getOrCreate()
df_video = spark.read.parquet('videos-preparados.snappy.parquet')
df_video.show(2)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|mukbang|378858|   18860|17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|   news|  6379|    4853|  808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+---

In [28]:
# Qtd total linhas col(Keyword)
count_keyword = df_video.groupBy('Keyword').count()
count_keyword.show()

+----------------+-----+
|         Keyword|count|
+----------------+-----+
|computer science|   48|
|            lofi|   40|
|         finance|   39|
|             cnn|   50|
|           apple|   42|
|            news|   39|
|         mukbang|   45|
|       education|   24|
|       interview|   50|
|          crypto|   50|
|   mathchemistry|   15|
|            food|   48|
|    data science|   50|
|        trolling|   50|
|        tutorial|   50|
|      literature|   46|
|             sat|   49|
|         history|   49|
|           cubes|   49|
|           music|   46|
+----------------+-----+
only showing top 20 rows


In [29]:
# Qtd valores unicos col(Keyword)
df_video.groupBy('Keyword').agg(
    format_number(avg('Interaction').alias('avg_interaction'),2)
).show()

+----------------+-----------------------------------------------------+
|         Keyword|format_number(avg(Interaction) AS avg_interaction, 2)|
+----------------+-----------------------------------------------------+
|computer science|                                         1,226,793.02|
|            lofi|                                         4,167,085.88|
|         finance|                                           708,542.95|
|             cnn|                                           570,650.86|
|           apple|                                        10,873,628.21|
|            news|                                           251,688.72|
|         mukbang|                                        11,053,630.38|
|       education|                                         2,750,838.62|
|       interview|                                         3,044,867.04|
|          crypto|                                           413,676.20|
|   mathchemistry|                                 

In [30]:
# Média Col(interaction) com groupBy col(Keyword)
df_video.groupBy('Keyword').agg(
    max('Interaction').alias('Rank Interactions')
).orderBy(desc('Rank Interactions')).show()

+--------+-----------------+
| Keyword|Rank Interactions|
+--------+-----------------+
| animals|       1593623628|
|   music|        922551152|
|     bed|        532691631|
| history|        440187490|
|   apple|        429916936|
| mrbeast|        300397699|
|  google|        239385460|
|business|        210025196|
|   cubes|        170925917|
|  sports|        106924567|
| mukbang|         87433858|
|    lofi|         86445177|
|tutorial|         69616442|
|  movies|         65253870|
|  marvel|         56247330|
|  how-to|         53053975|
|    food|         48754479|
| physics|         43463298|
|    asmr|         34411125|
|nintendo|         32268486|
+--------+-----------------+
only showing top 20 rows


In [31]:
# Média variancia Views
df_video.groupBy('Keyword').agg(
    avg('Views').alias('Média Views'),
    var_samp('Views').alias('Variancia Views')
).show()

+----------------+--------------------+--------------------+
|         Keyword|         Média Views|     Variancia Views|
+----------------+--------------------+--------------------+
|computer science|  1191958.7083333333| 2.81219868165102E12|
|            lofi|           4089363.0|1.846209641476677...|
|         finance|   694223.4358974359|3.304483175097042...|
|             cnn|           554240.38|1.563423618468118...|
|           apple|1.0746930452380951E7|4.299927977442589E15|
|            news|   247492.1794871795|1.067512576672564...|
|         mukbang|1.0904772355555555E7|5.586073238973179...|
|       education|  2684432.8333333335|1.833572249339214...|
|       interview|          2966111.86|1.819220996034335E13|
|          crypto|           404608.22|3.513691634369074E12|
|   mathchemistry|  3328125.2666666666|2.491467065256849...|
|            food|          5252406.25|7.326374128154842E13|
|    data science|           544771.98|5.479336525349994...|
|        trolling|      

In [32]:
# Média, Min e Max sem casas decimais
df_video.groupBy('Keyword').agg(
    format_number(avg('Views'),0).alias('Média Views'),
    min('Views').alias('Min Views'),
    max('Views').alias('Max Views')
).show()

+----------------+-----------+---------+---------+
|         Keyword|Média Views|Min Views|Max Views|
+----------------+-----------+---------+---------+
|computer science|  1,191,959|    16115|  7004107|
|            lofi|  4,089,363|     6817| 84747957|
|         finance|    694,223|     1195|  9450554|
|             cnn|    554,240|    51269|  1889320|
|           apple| 10,746,930|     1954|425478119|
|            news|    247,492|    10642|  1465011|
|         mukbang| 10,904,772|     3618| 86169225|
|       education|  2,684,433|     6611| 17103736|
|       interview|  2,966,112|     2587| 22529756|
|          crypto|    404,608|     1599| 11805668|
|   mathchemistry|  3,328,125|       25| 18496859|
|            food|  5,252,406|    47430| 48018833|
|    data science|    544,772|      911|  3069097|
|        trolling|  1,420,141|     5388| 14286302|
|        tutorial|  6,761,032|    19323| 68512549|
|      literature|    863,021|     2847|  4231789|
|             sat|  1,065,869| 

In [33]:
# MPrimeiro e último Published At
df_video.groupBy('Keyword').agg(
    first('Published At').alias('First Published At'),
    last('Published At').alias('Last Published At')
).orderBy('First Published At').show()

+----------------+------------------+-----------------+
|         Keyword|First Published At|Last Published At|
+----------------+------------------+-----------------+
|         biology|        2012-03-12|       2017-09-23|
|       education|        2015-02-06|       2010-10-14|
|      literature|        2016-08-05|       2016-01-15|
|         physics|        2017-01-02|       2016-01-24|
|          sports|        2017-12-04|       2020-12-28|
|        tutorial|        2018-06-01|       2022-08-17|
|    data science|        2019-08-18|       2021-08-06|
|         mukbang|        2020-04-18|       2022-08-24|
|   mathchemistry|        2020-08-11|       2019-10-04|
|         finance|        2020-09-23|       2017-12-31|
|          google|        2020-11-28|       2022-08-23|
|machine learning|        2021-02-12|       2017-11-01|
|       interview|        2021-08-03|       2018-10-05|
|game development|        2021-11-10|       2022-04-30|
|         mrbeast|        2021-11-24|       2021

In [34]:
# Contagem Title total e contagem Title primaryKey(valores únicos)
df_video.agg(count(col('Title'))).show()
df_video.agg(count_distinct(col('Title'))).show()

+------------+
|count(Title)|
+------------+
|        1869|
+------------+

+---------------------+
|count(DISTINCT Title)|
+---------------------+
|                 1854|
+---------------------+



In [35]:
# Todos os registros ordenados por ano
df_video.withColumn('Ano', year(col('Published At'))).groupBy('Ano').agg(
    count('Title').alias('Quantidade')
).orderBy(asc('Ano')).show()

+----+----------+
| Ano|Quantidade|
+----+----------+
|2007|         2|
|2008|         1|
|2009|         9|
|2010|         6|
|2011|         4|
|2012|        12|
|2013|         6|
|2014|        10|
|2015|        15|
|2016|        34|
|2017|        47|
|2018|        57|
|2019|        86|
|2020|       158|
|2021|       229|
|2022|      1193|
+----+----------+



In [46]:
# Ignore essa parte, apenas quis anexar de uma vez as colunas Ano e Mês na tabela principal
df_video = df_video.withColumn('Ano', year(col('Published At'))).withColumn('Mês', month(col('Published At')))
df_video.show()

+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+----+---+
|               Title|   Video ID|Published At|         Keyword| Likes|Comments|    Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features| Ano|Mês|
+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+----+---+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|         mukbang|378858|   18860| 17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|2020|  4|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|            news|  6379|    4853|   808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|2022|  8|


In [37]:
# Dupla ordenação, por ano e mês
df_video.groupBy('Ano', 'Mês').agg(
    count('Title').alias('Quantidade')
).orderBy('Ano', 'Mês').show()

+----+---+----------+
| Ano|Mês|Quantidade|
+----+---+----------+
|2007|  7|         1|
|2007| 12|         1|
|2008|  7|         1|
|2009|  2|         2|
|2009|  6|         2|
|2009|  7|         1|
|2009|  8|         1|
|2009| 10|         1|
|2009| 12|         2|
|2010|  3|         1|
|2010|  5|         2|
|2010|  6|         1|
|2010|  9|         1|
|2010| 10|         1|
|2011|  2|         1|
|2011|  5|         1|
|2011|  9|         1|
|2011| 10|         1|
|2012|  1|         1|
|2012|  2|         3|
+----+---+----------+
only showing top 20 rows


In [45]:
# Média Cumulativa col(Likes)
WindowSpec = Window.partitionBy('Keyword').orderBy('Ano').rowsBetween(Window.unboundedPreceding, Window.currentRow)

ordenacao = df_video.orderBy('Ano')
ordenacao = ordenacao.dropna()
print(df_video.count())

media_cumulativa_ano = ordenacao.withColumn('Média Likes Cumulativa', avg(col('Likes')).over(WindowSpec))
media_cumulativa_ano.select('Keyword', 'Ano', 'Likes', 'Média Likes Cumulativa').show()

1869
+-------+----+--------+----------------------+
|Keyword| Ano|   Likes|Média Likes Cumulativa|
+-------+----+--------+----------------------+
|animals|2009| 1357197|             1357197.0|
|animals|2010|   68133|              712665.0|
|animals|2010|  338601|              587977.0|
|animals|2013|11025176|            3197276.75|
|animals|2014| 1019385|             2761698.4|
|animals|2014| 5743875|    3258727.8333333335|
|animals|2019| 1103713|    2950868.5714285714|
|animals|2020|   74617|           2591337.125|
|animals|2020|    9313|    2304445.5555555555|
|animals|2020|  164337|             2090434.7|
|animals|2020|   94089|    1908948.7272727273|
|animals|2020|   21946|             1751698.5|
|animals|2020|   28863|    1619172.6923076923|
|animals|2020|   73362|     1508757.642857143|
|animals|2020|  282754|    1427024.0666666667|
|animals|2020| 6177588|          1723934.3125|
|animals|2021|   11323|    1623192.4705882352|
|animals|2021|   27102|    1534520.7777777778|
|animals